In [1]:
import yaml
import add_packages
from pprint import pprint
import os
import pandas as pd
from tqdm.auto import tqdm

from toolkit.langchain import (
	document_loaders, text_splitters, text_embedding_models, stores, 
	prompts, utils, output_parsers, agents, documents, models,
	runnables, tools, chains
)

from toolkit import sql, utils

PATH_DATA = f"{add_packages.APP_PATH}/data/tdtu/FEEE"
FILE_CFG = "tdtu.yaml"
tqdm.pandas(desc="Processing")

with open(f"{add_packages.APP_PATH}/my_configs/{FILE_CFG}", 'r') as file:
	configs = yaml.safe_load(file)

/opt/homebrew/Caskroom/miniconda/base/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
llm = models.chat_openai
embeddings = text_embedding_models.OpenAIEmbeddings()
vectorstore = stores.faiss.FAISS

In [ ]:
my_sql_db = sql.MySQLDatabase()
# my_sql_db = sql.MySQLDatabase(
# 	dbname=os.getenv("SQL_DB_NEON"),
# 	host=os.getenv("SQL_HOST_NEON"),
# 	port=os.getenv("SQL_PORT_NEON"),
# 	user=os.getenv("SQL_USER_NEON"),
# 	password=os.getenv("SQL_PASSWORD_NEON"),
# )

# Data

## txt

### tdtu_feee_faq

In [ ]:
path_txt = f"{PATH_DATA}/tdtu_feee_faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=1000, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "general information"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

In [ ]:
docs_txt_tdtu_feee_faq = docs_txt

### File 2

In [ ]:
path_txt = f"{PATH_DATA}/faq.txt"

In [ ]:
loader_txt = document_loaders.TextLoader(path_txt)
doc_txt = loader_txt.load()

text_splitter = text_splitters.RecursiveCharacterTextSplitter(
	# chunk_size=500, chunk_overlap=100,
	separators=["##"], chunk_size=150, chunk_overlap=0,
)
docs_txt = text_splitter.split_documents(doc_txt)
docs_txt = docs_txt[1:]

metadatas = {
	"data": "frequently asked questions"
}
utils.remove_metadata(docs_txt, "source")
utils.update_metadata(docs_txt, metadatas)

## csv

### NhanSu

In [ ]:
file_xlsx = "NhanSu.xlsx"
path_xlsx = f"{PATH_DATA}/{file_xlsx}"
path_xlsx_processed = f"{PATH_DATA}/{file_xlsx.split('.')[0]}1.xlsx"

In [ ]:
df = pd.read_excel(
	path_xlsx, 
 	# delimiter=";"
)

df.head()

# Prompting to get new col names

#### Process

In [ ]:
model = models.chat_openai

template1 = """\
...
{text}
"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

# chain = runnables.RunnablePassthrough.assign(
#   text=chain1
# ).assign(
#   text=chain2
# )


chain = runnables.RunnablePassthrough.assign(
  text=chain1
)

def process_xlsx_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

def capitalize_first_letter(s):
	return ' '.join([word.capitalize() for word in s.split()])

def change_col_value(df: pd.DataFrame, column_name: str, value, new_value):
	df[column_name] = df[column_name].replace(value, new_value)
	return df

def replace_col_value_if_contains(df, column_name, substring, new_substring):
	df[column_name] = df[column_name].str.replace(substring, new_substring)
	return df

# query = '...'
# result = process_xlsx_col(query)

# pprint(result)

In [ ]:
col_to_process = "Liên hệ"

df[col_to_process] = df[col_to_process].progress_apply(process_xlsx_col)


In [ ]:
# df.to_excel(f"{path_xlsx_processed}", index=False)

In [ ]:
path_xlsx = path_xlsx_processed

#### Load to sql

In [ ]:
my_table_schema = [
	"id SERIAL",
	"faculty TEXT",
	"name TEXT",
	"position TEXT",
	"major TEXT",
	"email TEXT",
	"office TEXT",
	"child_department TEXT",
	"PRIMARY KEY (id)",
]

my_table = sql.MySQLTable(
	name="tdtu_feee_personnel", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

db = stores.SQLDatabase.from_uri(my_sql_db.get_uri())

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
# my_table.insert_from_dataframe(df)

In [ ]:
# df = pd.read_excel(path_xlsx)
# df.columns = table_cols

# my_table.insert_from_dataframe(df)

# cols = ['name', 'position', 'major', 'office', 'child_department']
# proper_nouns = [value for col in cols for value in my_table.get_discrete_values_col(col)]


In [ ]:
# all_proper_nouns.extend(proper_nouns)

In [ ]:
questions = ...
examples_questions_to_sql = ...

### ChuongTrinhDaoTao

In [ ]:
file_xlsx = "ChuongTrinhDaoTao.xlsx"
path_xlsx = f"{PATH_DATA}/{file_xlsx}"
path_xlsx_processed = f"{PATH_DATA}/{file_xlsx.split('.')[0]}1.xlsx"

In [ ]:
df = pd.read_excel(
	path_xlsx, 
 	# delimiter=";"
)

df.head()

# Prompting to get new col names

#### Process

In [ ]:
model = models.chat_openai

template1 = """\
...
{text}
"""

template2 = """\
...
{text}
"""

prompt_template1 = prompts.PromptTemplate.from_template(template1)
prompt_template2 = prompts.PromptTemplate.from_template(template2)

chain1 = prompt_template1 | model | output_parsers.StrOutputParser()
chain2 = prompt_template2 | model | output_parsers.StrOutputParser()

# chain = runnables.RunnablePassthrough.assign(
#   text=chain1
# ).assign(
#   text=chain2
# )


chain = runnables.RunnablePassthrough.assign(
  text=chain1
)

def process_xlsx_col(text: str) -> str:
  result = chain.invoke({"text": text})['text']
  return result

def capitalize_first_letter(s):
	return ' '.join([word.capitalize() for word in s.split()])

def change_col_value(df: pd.DataFrame, column_name: str, value, new_value):
	df[column_name] = df[column_name].replace(value, new_value)
	return df

def replace_col_value_if_contains(df, column_name, substring, new_substring):
	df[column_name] = df[column_name].str.replace(substring, new_substring)
	return df

# query = '...'
# result = process_xlsx_col(query)

# pprint(result)

In [ ]:
col_to_process = "Liên hệ"

df[col_to_process] = df[col_to_process].progress_apply(process_xlsx_col)


In [ ]:
df.to_excel(f"{path_xlsx_processed}", index=False)

In [ ]:
df

In [ ]:
path_xlsx = path_xlsx_processed

#### Load to sql

In [ ]:
my_table_schema = [
	"id SERIAL",
	"faculty TEXT",
	"study_field TEXT",
	"link TEXT",
	"program_type TEXT",
	"education_level TEXT",
	"introduction TEXT",
	"career_prospects TEXT",
	"outcome TEXT",
	"syllabub TEXT",
	"admission_candidates TEXT",
	"registration TEXT",
	"tuition TEXT",
 	"contact TEXT",
	"PRIMARY KEY (id)",
]
my_table = sql.MySQLTable(
	name="tdtu_feee_admission", 
	schema=my_table_schema,
	db=my_sql_db,
)
my_table.create()

table_cols = [col_description.split(" ")[0] for col_description in my_table_schema][1:-1]


In [ ]:
# my_table.insert_from_dataframe(df)

In [ ]:
# df = pd.read_excel(path_xlsx)
# df.columns = table_cols

# my_table.insert_from_dataframe(df)

# cols = ['faculty', 'study_field', 'program_type', 'education_level']
# proper_nouns = [value for col in cols for value in my_table.get_discrete_values_col(col)]


In [ ]:
# all_proper_nouns.extend(proper_nouns)

In [ ]:
questions = ...
examples_questions_to_sql = ...

# Vector store 

## txt

### tdtu_feee_faq

In [ ]:
qdrant_txt_tdtu_feee_faq = stores.QdrantStore(
  embeddings_provider="openai",
	embeddings_model="text-embedding-3-large",
	llm=models.chat_openai,
	search_type="mmr",
  configs=configs,
  distance="Cosine",
  retriever_types="base",
  **configs["vector_db"]["qdrant"]["tdtu_feee_faq"],
)

In [ ]:
# qdrant_txt_tdtu_feee_faq.add_documents(docs_txt_tdtu_feee_faq)

# Test

In [ ]:
my_chain_rag_tdtu_feee_faq = chains.MyRagChain(
	llm=llm,
	retriever=qdrant_txt_tdtu_feee_faq.retriever,
	is_debug=False,
	just_return_ctx=True,
	**configs["vector_db"]["qdrant"]["tdtu_feee_faq"],
)

tool_chain_rag_tdtu_feee_faq = my_chain_rag_tdtu_feee_faq.create_tool_chain_rag()


In [ ]:
examples_fewshot_tmp = dict(configs["sql"]["examples_questions_to_sql"]).values()
examples_questions_to_sql = [example for sublist in examples_fewshot_tmp for example in sublist]

proper_nouns = configs["sql"]["proper_nouns"]

my_sql_db = sql.MySQLDatabase()

cfg_sql = configs["sql"]
cfg_sql_tool = cfg_sql["tool"]

my_sql_chain = chains.MySqlChain(
	my_sql_db=my_sql_db,
	llm=llm,
	embeddings=embeddings,
	vectorstore=vectorstore,
	proper_nouns=proper_nouns,
	k_retriever_proper_nouns=4,
	examples_questions_to_sql=examples_questions_to_sql,
	k_few_shot_examples=5,
	sql_max_out_length=2000,
	is_sql_get_all=True,
	is_debug=False,
	tool_name=cfg_sql_tool["name"],
	tool_description=cfg_sql_tool["description"],
	tool_metadata=cfg_sql_tool["metadata"],
	tool_tags=cfg_sql_tool["tags"],
)

tool_chain_sql = my_sql_chain.create_tool_chain_sql()

In [ ]:
llm = models.create_llm(provider="openai", version="gpt-3.5-turbo-0125")

tools = [
	tool_chain_rag_tdtu_feee_faq,
	tool_chain_sql,
]

system_message_custom = configs["prompts"]["system_message_tdtu"]
prompt = prompts.create_prompt_tool_calling_agent(system_message_custom)

agent = agents.MyStatelessAgent(
	llm=llm,
	tools=tools,
	prompt=prompt,
	agent_type="tool_calling",
	agent_verbose=False,
)

In [ ]:
res = []
async for chunk in agent.astream_events_basic(
	# "Người phụ trách bộ môn Điều khiển tự động khoa Điện",
	# "Các tiến sĩ trong khoa Điện", # adjust prompt to return all result
	# "Các thạc sĩ trong khoa Điện",
	"Ký túc xá",
  show_tool_call=True,
  history_type="mongodb",
  user_id=utils.generate_unique_id(thing="uuid_name"),
	session_id=utils.generate_unique_id(thing="uuid"),
):
	print(chunk, end="", flush=True)

	res.append(chunk)

In [ ]:
result = my_sql_chain.chain_sql.invoke({
	"question": 
   	"Các chương trình đào tạo khoa Điện có hình thức liên kết với các trường đại học nước ngoài?",
})

result